In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import cv2 as cv
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/leaf_dis.zip

Archive:  /content/drive/MyDrive/leaf_dis.zip
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0365.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0366.jpg  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0367.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0370.jpg  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0372.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0373.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0374.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0375.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0376.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0377.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0378.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0379.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC_0380.JPG  
  inflating: rice_leaf_diseases/Bacterial leaf blight/DSC

In [ ]:
input_dir = '/content/rice_leaf_diseases'
output_dir_resized = '/content/rice_leaf_diseases_resized'
output_dir_downsampled = '/content/rice_leaf_diseases_downsampled'

In [ ]:
# Create output directories if they don't exist
os.makedirs(output_dir_resized, exist_ok=True)
os.makedirs(output_dir_downsampled, exist_ok=True)

target_width = 224
target_height = 224
downsample_factor = 0.5  # Reducing image dimensions by 50%


In [ ]:
# Iterate through each category
for category in os.listdir(input_dir):
    category_path = os.path.join(input_dir, category)
    resized_category_path = os.path.join(output_dir_resized, category)
    downsampled_category_path = os.path.join(output_dir_downsampled, category)
    os.makedirs(resized_category_path, exist_ok=True)
    os.makedirs(downsampled_category_path, exist_ok=True)

    # Iterate through images in the category
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = Image.open(img_path)

        # Resize image to the target width and height
        resized_img = img.resize((target_width, target_height), Image.ANTIALIAS)

        # Save resized image
        resized_img_path = os.path.join(resized_category_path, img_name)
        resized_img.save(resized_img_path)

        # Downsample image
        downsampled_img = img.resize((int(img.width * downsample_factor), int(img.height * downsample_factor)), Image.ANTIALIAS)

        # Save downsampled image
        downsampled_img_path = os.path.join(downsampled_category_path, img_name)
        downsampled_img.save(downsampled_img_path)

<ipython-input-17-40b9f0c3eb02>:15: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize((target_width, target_height), Image.ANTIALIAS)
<ipython-input-17-40b9f0c3eb02>:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  downsampled_img = img.resize((int(img.width * downsample_factor), int(img.height * downsample_factor)), Image.ANTIALIAS)


In [ ]:
disease = os.listdir('/content/rice_leaf_diseases_resized')
main_list = []
for i in range(len(disease)):
    sub_list=[]
    os.chdir('/content/rice_leaf_diseases_resized/{}'.format(disease[i]))
    imgs = os.listdir()
    for j in imgs:
        img = cv.imread(j,-1).flatten()
        sub_list.append([img,disease[i]])
    main_list.append(sub_list)

In [ ]:
len(main_list)

3

In [ ]:
len(main_list[1][0][0])

150528

In [ ]:
df_main_list = []
for i in range(3):
    df = pd.DataFrame(main_list[i])
    df_main_list.append(df)
df = pd.concat(df_main_list,axis=0,ignore_index=True)
df = pd.concat([df[0].apply(pd.Series),df[1]],axis=1,ignore_index=True) # Explode the lists
# shuffle all the  rows
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,150519,150520,150521,150522,150523,150524,150525,150526,150527,150528
0,233,235,245,233,235,245,233,235,245,233,...,231,233,243,231,233,243,231,233,243,Brown spot
1,226,231,240,226,231,240,226,231,240,226,...,212,222,239,212,222,239,212,222,239,Bacterial leaf blight
2,229,233,238,229,233,238,229,233,238,229,...,224,228,239,224,228,239,224,228,239,Bacterial leaf blight
3,234,237,242,234,237,242,234,237,242,234,...,227,232,241,227,232,241,227,232,241,Leaf smut
4,221,224,238,221,224,238,221,224,238,221,...,218,221,236,218,221,236,219,222,237,Leaf smut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,230,232,242,230,232,242,230,232,242,229,...,217,225,238,217,225,238,217,225,238,Bacterial leaf blight
116,212,215,230,213,216,231,215,218,233,216,...,216,224,237,215,223,236,214,222,235,Brown spot
117,235,238,243,235,238,243,235,238,243,235,...,230,234,239,230,234,239,230,234,239,Bacterial leaf blight
118,220,229,239,220,229,239,220,229,239,220,...,217,225,238,217,225,238,218,226,239,Leaf smut


In [ ]:
df1 = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df1

,0,1,2,3,4,5,6,7,8,9,...,150519,150520,150521,150522,150523,150524,150525,150526,150527,150528
0,230,234,239,230,234,239,230,234,239,230,...,224,230,243,224,230,243,224,230,243,Brown spot
1,67,175,216,53,160,198,21,126,159,0,...,255,255,255,255,255,255,255,255,255,Brown spot
2,225,229,240,225,229,240,225,229,240,225,...,222,231,241,222,231,241,222,231,241,Brown spot
3,230,232,242,230,232,242,230,232,242,230,...,218,227,237,218,227,237,218,227,237,Leaf smut
4,220,228,241,219,227,240,219,227,240,218,...,218,226,239,218,226,239,218,226,239,Leaf smut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,59,181,205,62,184,208,67,186,208,68,...,38,119,110,38,119,110,39,120,111,Brown spot
116,230,232,242,228,230,240,226,228,238,226,...,226,228,239,226,228,239,226,228,239,Leaf smut
117,230,234,239,230,234,239,230,234,239,230,...,229,231,241,229,231,241,229,231,241,Bacterial leaf blight
118,226,228,239,226,228,239,226,228,239,226,...,218,226,239,218,226,239,218,226,239,Brown spot


In [ ]:
x=df1.iloc[:,:-1]

In [ ]:
x

,0,1,2,3,4,5,6,7,8,9,...,150518,150519,150520,150521,150522,150523,150524,150525,150526,150527
0,230,234,239,230,234,239,230,234,239,230,...,243,224,230,243,224,230,243,224,230,243
1,67,175,216,53,160,198,21,126,159,0,...,255,255,255,255,255,255,255,255,255,255
2,225,229,240,225,229,240,225,229,240,225,...,241,222,231,241,222,231,241,222,231,241
3,230,232,242,230,232,242,230,232,242,230,...,238,218,227,237,218,227,237,218,227,237
4,220,228,241,219,227,240,219,227,240,218,...,239,218,226,239,218,226,239,218,226,239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,59,181,205,62,184,208,67,186,208,68,...,109,38,119,110,38,119,110,39,120,111
116,230,232,242,228,230,240,226,228,238,226,...,239,226,228,239,226,228,239,226,228,239
117,230,234,239,230,234,239,230,234,239,230,...,241,229,231,241,229,231,241,229,231,241
118,226,228,239,226,228,239,226,228,239,226,...,239,218,226,239,218,226,239,218,226,239


In [ ]:
y=df1.iloc[:,-1]

In [ ]:
y

0                 Brown spot
1                 Brown spot
2                 Brown spot
3                  Leaf smut
4                  Leaf smut
               ...          
115               Brown spot
116                Leaf smut
117    Bacterial leaf blight
118               Brown spot
119    Bacterial leaf blight
Name: 150528, Length: 120, dtype: object

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(x,y,test_size=0.15)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'entropy', random_state =47)
tree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=47)

In [ ]:
y_pred = tree.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

array([[4, 2, 0],
       [0, 6, 2],
       [1, 0, 3]])

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7222222222222222
